In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import os
import warnings
import functools
import seaborn as sns
import scipy.stats
import anndata
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

warnings.filterwarnings('ignore')
os.chdir(os.path.expanduser('/home/jovyan/Prostate_analysis/scanpy'))
sc.settings.verbosity = 3
sc.logging.print_versions()

results_file = 'out/mnp.integrated.h5ad'

adata = sc.read_h5ad(results_file)
adata

/home/jovyan/my-conda-envs/mypython3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


scanpy==1.4.5.post2 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.2 scipy==1.4.1 pandas==0.25.1 scikit-learn==0.22.1 statsmodels==0.11.0rc1 python-igraph==0.7.1 louvain==0.6.1


AnnData object with n_obs × n_vars = 793 × 3000 
    obs: 'age', 'barcode', 'batch', 'group', 'highest_GLEASON_score', 'mri_grading', 'name', 'patient', 'pool', 'psa', 'sample', 'scrublet_score', 'scrublet_cluster_score', 'bh_pval', 'is_doublet', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'leiden', 'celltype', 'cohort', 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'celltype-immune'
    var: 'gene_ids', 'feature_types'
    uns: 'celltype-immune_colors', 'cohort_colors', 'dendrogram_celltype-immune', 'dendrogram_leiden', 'group_colors', 'leiden', 'leiden_colors', 'neighbors', 'patient_colors', 'pca', 'phase_colors', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [2]:
### to perform GSEA of mnp subtypes, first do a wilcoxon test between them
ndata = adata[adata.obs['group'] =='normal']
sc.tl.rank_genes_groups(ndata, groupby = 'celltype-immune', n_genes = 30000, method = 'wilcoxon')

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


In [3]:
def exportDEres(adata, column, filename):
    scores = pd.DataFrame(data = adata.uns['rank_genes_groups']['scores'][column], index = adata.uns['rank_genes_groups']['names'][column])
    lfc = pd.DataFrame(data = adata.uns['rank_genes_groups']['logfoldchanges'][column], index = adata.uns['rank_genes_groups']['names'][column])
    pvals = pd.DataFrame(data = adata.uns['rank_genes_groups']['pvals'][column], index = adata.uns['rank_genes_groups']['names'][column])
    padj = pd.DataFrame(data = adata.uns['rank_genes_groups']['pvals_adj'][column], index = adata.uns['rank_genes_groups']['names'][column])
    dfs = [scores, lfc, pvals, padj]
    df_final = functools.reduce(lambda left,right: pd.merge(left,right,left_index = True, right_index = True), dfs)
    df_final.columns = ['scores', 'logfoldchanges', 'pvals', 'pvals_adj']

    df_final.to_csv(filename, sep = '\t')

outpath = 'out/DEG/'
if not os.path.exists(outpath):
        os.makedirs(outpath)
for x in list(set(adata.obs['celltype-immune'])):
    exportDEres(ndata, str(x), outpath+str(x)+'_normal.txt')

In [5]:
def exportDEres(adata, column, filename):
    scores = pd.DataFrame(data = adata.uns['rank_genes_groups_filtered']['scores'][column], index = adata.uns['rank_genes_groups_filtered']['names'][column])
    lfc = pd.DataFrame(data = adata.uns['rank_genes_groups_filtered']['logfoldchanges'][column], index = adata.uns['rank_genes_groups_filtered']['names'][column])
    pvals = pd.DataFrame(data = adata.uns['rank_genes_groups_filtered']['pvals'][column], index = adata.uns['rank_genes_groups_filtered']['names'][column])
    padj = pd.DataFrame(data = adata.uns['rank_genes_groups_filtered']['pvals_adj'][column], index = adata.uns['rank_genes_groups_filtered']['names'][column])
    scores = scores.loc[scores.index.dropna()]
    lfc = lfc.loc[lfc.index.dropna()]
    pvals = pvals.loc[pvals.index.dropna()]
    padj = padj.loc[padj.index.dropna()]
    dfs = [scores, lfc, pvals, padj]
    df_final = functools.reduce(lambda left,right: pd.merge(left,right,left_index = True, right_index = True), dfs)
    df_final.columns = ['scores', 'logfoldchanges', 'pvals', 'pvals_adj']

    df_final.to_csv(filename, sep = '\t')
# also do the within celltype tumor vs normal DEGs 
for x in list(set(adata.obs['celltype-immune'])):
    xdata = adata[adata.obs['celltype-immune'] == x]
    sc.tl.rank_genes_groups(xdata, groupby = 'group', n_genes = 30000, method = 'wilcoxon')
    sc.tl.filter_rank_genes_groups(xdata, min_in_group_fraction=0.25, min_fold_change=1, max_out_group_fraction=0.5)
    exportDEres(xdata, 'tumor', outpath+str(x)+'_tumor_vs_normal.txt')

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
Filtering genes using: min_in_group_fraction: 0.25 min_fold_change: 1, max_out_group_fraction: 0.5
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
Filtering genes using: min_in_group_fraction: 0.25 min_fold_change: 1, max_out_group_fraction: 0.5
ranking genes
    finished: added 

In [ ]:
# ### run GSEA on the tumor vs normal?
# def rankList(deg_file, remove_ribo_mito = False):
#     # read in the files and do some ranking calculations
#     deg = pd.read_csv(deg_file, sep = '\t', index_col = 0)
#     if remove_ribo_mito:
#         # remove ribosomal and mitochondrial genes
#         deg = deg[~deg.index.str.contains('RPS|RPL|MT-')]

#     # convert to negative log10 pval
#     deg['neglog10pval'] = [-1*np.log10(p) for p in deg['pvals']]
#     # convert inf values to max machine number
#     try:
#         deg['neglog10pval'].replace(np.inf, -1*np.log10(10**-308), inplace = True)
#     except:
#         pass
#     try:    
#         deg['neglog10pval'].replace(-np.inf, -1*np.log10(10**308), inplace = True)
#     except:
#         pass
#     deg['rank'] = [p*np.sign(lfc) for p, lfc in zip(deg['neglog10pval'], deg['logfoldchanges'])]
#     deg = deg.reset_index(drop = False)
#     return(deg[['index', 'rank']])
# # quickly prep the files
# DC = rankList('out/DEG/DC_tumor_vs_normal.txt', True)
# Mac1 = rankList('out/DEG/Mac1_tumor_vs_normal.txt', True)
# Mac2 = rankList('out/DEG/Mac2_tumor_vs_normal.txt', True)
# MacMT1 = rankList('out/DEG/Mac-MT1_tumor_vs_normal.txt', True)
# Mono = rankList('out/DEG/Mono_tumor_vs_normal.txt', True)

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import os
import warnings
import functools
import seaborn as sns
import scipy.stats
import anndata
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

warnings.filterwarnings('ignore')
os.chdir(os.path.expanduser('/home/jovyan/Prostate_analysis/scanpy'))
sc.settings.verbosity = 3
sc.logging.print_versions()

results_file = 'out/lymphoid.h5ad'

adata = sc.read_h5ad(results_file)
adata

In [ ]:
### do a wilcoxon test between the lymphoid cells, excluding the B cells
ndata = adata[adata.obs['group'] =='normal']
sc.tl.rank_genes_groups(ndata, groupby = 'celltype-immune', n_genes = 30000, method = 'wilcoxon')

In [ ]:
def exportDEres(adata, column, filename):
    scores = pd.DataFrame(data = adata.uns['rank_genes_groups']['scores'][column], index = adata.uns['rank_genes_groups']['names'][column])
    lfc = pd.DataFrame(data = adata.uns['rank_genes_groups']['logfoldchanges'][column], index = adata.uns['rank_genes_groups']['names'][column])
    pvals = pd.DataFrame(data = adata.uns['rank_genes_groups']['pvals'][column], index = adata.uns['rank_genes_groups']['names'][column])
    padj = pd.DataFrame(data = adata.uns['rank_genes_groups']['pvals_adj'][column], index = adata.uns['rank_genes_groups']['names'][column])
    dfs = [scores, lfc, pvals, padj]
    df_final = functools.reduce(lambda left,right: pd.merge(left,right,left_index = True, right_index = True), dfs)
    df_final.columns = ['scores', 'logfoldchanges', 'pvals', 'pvals_adj']

    df_final.to_csv(filename, sep = '\t')

outpath = 'out/DEG/'
if not os.path.exists(outpath):
        os.makedirs(outpath)
for x in list(set(adata.obs['celltype-immune'])):
    exportDEres(ndata, str(x), outpath+str(x).replace('/','_')+'_normal.txt')

In [ ]:
# also do the within celltype tumor vs normal DEGs
for x in list(set(adata.obs['celltype-immune'])):
    xdata = adata[adata.obs['celltype-immune'] == x]
    sc.tl.rank_genes_groups(xdata, groupby = 'group', n_genes = 30000, method = 'wilcoxon')
    exportDEres(xdata, 'tumor', outpath+str(x).replace('/','_')+'_tumor_vs_normal.txt')

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import os
import warnings
import functools
import seaborn as sns
import scipy.stats
import anndata
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

warnings.filterwarnings('ignore')
os.chdir(os.path.expanduser('/home/jovyan/Prostate_analysis/scanpy'))
sc.settings.verbosity = 3
sc.logging.print_versions()

results_file = 'out/mnp.integrated.h5ad'

adata = sc.read_h5ad(results_file)
adata

In [ ]:
### to perform GSEA of mnp subtypes, first do a wilcoxon test between them
sc.tl.rank_genes_groups(adata, groupby = 'celltype-immune', n_genes = 30000, method = 'wilcoxon')

In [ ]:
def exportDEres(adata, column, filename):
    scores = pd.DataFrame(data = adata.uns['rank_genes_groups']['scores'][column], index = adata.uns['rank_genes_groups']['names'][column])
    lfc = pd.DataFrame(data = adata.uns['rank_genes_groups']['logfoldchanges'][column], index = adata.uns['rank_genes_groups']['names'][column])
    pvals = pd.DataFrame(data = adata.uns['rank_genes_groups']['pvals'][column], index = adata.uns['rank_genes_groups']['names'][column])
    padj = pd.DataFrame(data = adata.uns['rank_genes_groups']['pvals_adj'][column], index = adata.uns['rank_genes_groups']['names'][column])
    dfs = [scores, lfc, pvals, padj]
    df_final = functools.reduce(lambda left,right: pd.merge(left,right,left_index = True, right_index = True), dfs)
    df_final.columns = ['scores', 'logfoldchanges', 'pvals', 'pvals_adj']

    df_final.to_csv(filename, sep = '\t')

outpath = 'out/DEG/'
if not os.path.exists(outpath):
        os.makedirs(outpath)
for x in list(set(adata.obs['celltype-immune'])):
    exportDEres(adata, str(x), outpath+str(x)+'_normalandtumor.txt')

In [ ]:
### run GSEA on Mac-MT1 for finding the leading edge of the lipid pathways
def rankList(deg_file, remove_ribo_mito = False):
    # read in the files and do some ranking calculations
    deg = pd.read_csv(deg_file, sep = '\t', index_col = 0)
    if remove_ribo_mito:
        # remove ribosomal and mitochondrial genes
        deg = deg[~deg.index.str.contains('RPS|RPL|MT-')]

    # convert to negative log10 pval
    deg['neglog10pval'] = [-1*np.log10(p) for p in deg['pvals']]
    # convert inf values to max machine number
    try:
        deg['neglog10pval'].replace(np.inf, -1*np.log10(10**-308), inplace = True)
    except:
        pass
    try:    
        deg['neglog10pval'].replace(-np.inf, -1*np.log10(10**308), inplace = True)
    except:
        pass
    deg['rank'] = [p*np.sign(lfc) for p, lfc in zip(deg['neglog10pval'], deg['logfoldchanges'])]
    deg = deg.reset_index(drop = False)
    return(deg[['index', 'rank']])
# quickly prep the files
MacMT1 = rankList('out/DEG/Mac-MT1_normal.txt', True)

In [ ]:
# run preranked gsea
import gseapy as gp
pre_res = gp.prerank(rnk=MacMT1,
                     gene_sets='dataset/Macrophage_stim_markers_Human.gmt',
                     processes=10,
                     min_size=0,
                     max_size=1000,
                     permutation_num=1000, # reduce number to speed up testing
                     no_plot=True,
                     verbose=True)
pre_res.res2d.to_csv('out/GSEA/Mac-MT1_normal_macstim_gsea.txt', sep ='\t')

In [ ]:
# run preranked gsea
import gseapy as gp
pre_res = gp.prerank(rnk=MacMT1,
                     gene_sets='dataset/Hallmarks_metabolism_genesets.gmt',
                     processes=10,
                     min_size=0,
                     max_size=1000,
                     permutation_num=1000, # reduce number to speed up testing
                     no_plot=True,
                     verbose=True)
pre_res.res2d.to_csv('out/GSEA/Mac-MT1_normal_hallmark_metabolism_gsea.txt', sep ='\t')

In [ ]:
### tumor vs normal
import pandas as pd
import numpy as np
### run GSEA on the tumor vs normal on Mac-MT1 for just the Hallmark metabolism genes
def rankList(deg_file, remove_ribo_mito = False):
    # read in the files and do some ranking calculations
    deg = pd.read_csv(deg_file, sep = '\t', index_col = 0)
    if remove_ribo_mito:
        # remove ribosomal and mitochondrial genes
        deg = deg[~deg.index.str.contains('RPS|RPL|MT-')]

    # convert to negative log10 pval
    deg['neglog10pval'] = [-1*np.log10(p) for p in deg['pvals']]
    # convert inf values to max machine number
    try:
        deg['neglog10pval'].replace(np.inf, -1*np.log10(10**-308), inplace = True)
    except:
        pass
    try:    
        deg['neglog10pval'].replace(-np.inf, -1*np.log10(10**308), inplace = True)
    except:
        pass
    deg['rank'] = [p*np.sign(lfc) for p, lfc in zip(deg['neglog10pval'], deg['logfoldchanges'])]
    deg = deg.reset_index(drop = False)
    return(deg[['index', 'rank']])
# quickly prep the files
MacMT1 = rankList('out/DEG/Mac-MT1_tumor_vs_normal.txt', True)

In [ ]:
# run preranked gsea
import gseapy as gp
pre_res = gp.prerank(rnk=MacMT1,
                     gene_sets='dataset/Hallmarks_metabolism_genesets.gmt',
                     processes=10,
                     min_size=0,
                     max_size=1000,
                     permutation_num=1000, # reduce number to speed up testing
                     no_plot=True,
                     verbose=True)
pre_res.res2d.to_csv('out/GSEA/Mac-MT1_tumor_vs_normal_hallmark_metabolism_gsea.txt', sep ='\t')

In [ ]:
pre_res = gp.prerank(rnk=MacMT1,
                     gene_sets='dataset/h.all.v7.0.symbols.gmt',
                     processes=10,
                     min_size=0,
                     max_size=1000,
                     permutation_num=1000, # reduce number to speed up testing
                     no_plot=True,
                     verbose=True)
pre_res.res2d.to_csv('out/GSEA/Mac-MT1_tumor_vs_normal_hallmarks_gsea.txt', sep ='\t')

In [ ]:
pre_res = gp.prerank(rnk=MacMT1,
                     gene_sets='dataset/Macrophage_stim_markers_Human.gmt',
                     processes=10,
                     min_size=0,
                     max_size=1000,
                     permutation_num=1000, # reduce number to speed up testing
                     no_plot=True,
                     verbose=True)
pre_res.res2d.to_csv('out/GSEA/Mac-MT1_tumor_vs_normal_macstim_gsea.txt', sep ='\t')